In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data
import yfinance as yf

In [3]:
start = '2010-01-01'
end = '2022-12-31'
yf.pdr_override()
df = data.DataReader('AAPL', start, end)
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04 00:00:00-05:00,7.622500,7.660714,7.585000,7.643214,6.515213,493729600
2010-01-05 00:00:00-05:00,7.664286,7.699643,7.616071,7.656429,6.526477,601904800
2010-01-06 00:00:00-05:00,7.656429,7.686786,7.526786,7.534643,6.422665,552160000
2010-01-07 00:00:00-05:00,7.562500,7.571429,7.466071,7.520714,6.410790,477131200
2010-01-08 00:00:00-05:00,7.510714,7.571429,7.466429,7.570714,6.453411,447610800


In [ ]:
df.tail()

In [ ]:
df = df.reset_index()
df.head()

In [ ]:
df = df.drop(['Date', 'Adj Close'], axis=1)
df.head()

In [ ]:
plt.plot(df['Close'])

In [ ]:
ma100 =df.Close.rolling(100).mean()
ma100

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.Close)
plt.plot(ma100, 'r')

In [ ]:
ma200 =df.Close.rolling(200).mean()
ma200

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.Close)
plt.plot(ma100, 'r', label='100 days moving average')
plt.plot(ma200, 'g', label='200 days moving average')
plt.legend()

In [ ]:
df.shape

In [ ]:
# Splitting Data into Training and Testing
data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.7)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.7):int(len(df))])


print(data_training.shape)
print(data_testing.shape)

In [ ]:
data_training.head()

In [ ]:
data_testing.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_training_array = scaler.fit_transform(data_training)
data_training_array

In [ ]:
x_train =[]
y_train =[]

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100:i])
    y_train.append(data_training_array[i,0])
    
x_train, y_train = np.array(x_train), np.array(y_train)    
    

In [ ]:
x_train.shape[1:]

In [ ]:
#ML Model

In [ ]:

from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential



In [ ]:
model = Sequential()
model.add(LSTM(units = 50, activation = 'relu', return_sequences =True, input_shape=(x_train.shape[1:])))
model.add(Dropout(0.2)) 

model.add(LSTM(units = 60, activation = 'relu', return_sequences =True))
model.add(Dropout(0.3)) 
          
model.add(LSTM(units = 80, activation = 'relu', return_sequences =True))
model.add(Dropout(0.4)) 
          
model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5)) 

model.add(Dense(units=1))
          

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=50)

In [ ]:
model.save('keras_model.h5')

In [ ]:
data_testing.head()

In [ ]:
past_100_days = data_training.tail(100)

In [ ]:
final_df = past_100_days.append(data_testing, ignore_index = True)

In [ ]:
input_data = scaler.fit_transform(final_df)
input_data

In [ ]:
input_data.shape

In [ ]:
A= [i for i in range(100, 105)]
A

In [ ]:
x_test = []
y_test = []
for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i-100:i])
    y_test.append(input_data[i,0])
x_test, y_test = np.array(x_test), np.array(y_test)

print(x_test.shape)
print(y_test.shape)

In [ ]:
#Making Prediction
y_pred = model.predict(x_test)

In [ ]:
y_pred.shape

In [ ]:
scale_factor = 1/scaler.scale_
y_pred*=scale_factor
y_test*=scale_factor

In [ ]:
plt.figure(figsize = (12,6))
plt.plot(y_test, 'b', label='Original Price')
plt.plot(y_pred, 'r', label='Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()